In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import copy, random, warnings

warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv("network_edgelist.csv").dropna()
weight_edge_all = copy.deepcopy(df)
df["target_hate"] = df["target"].apply(lambda s: int(s.split("__")[1]) < 1000000)
df["source_sns"] = df["source"].apply(lambda s: s.split("__")[0])
df["target_sns"] = df["target"].apply(lambda s: s.split("__")[0])

In [ ]:
s_node = df[["source", "source_sns"]]
t_node = df[["target", "target_sns"]]
s_node = s_node.drop_duplicates(keep="first")
t_node = t_node.drop_duplicates(keep="first")
s_node.columns = ["id", "type"]
t_node.columns = ["id", "type"]
node = s_node._append(t_node)
node = node.drop_duplicates(subset=["id"], keep="first")
n_count = node.groupby(["type"])["id"].count().reset_index(name="count")

In [ ]:
df_hate = df[df["target_hate"]]
snode = df_hate[["source", "source_sns"]].drop_duplicates(keep="first")
tnode = df_hate[["target", "target_sns"]].drop_duplicates(keep="first")
snode.columns = ["id", "type"]
tnode.columns = ["id", "type"]
Hate_node = snode._append(tnode)
Hate_node = Hate_node.drop_duplicates(subset=["id"], keep="first")

In [ ]:
hate_list = list(Hate_node["id"])
noHate_node = node[~node["id"].isin(hate_list)]
nohate_list = list(set(noHate_node["id"]))

In [ ]:
Hate_node["sign"] = np.arange(1, Hate_node.shape[0] + 1, 1)
noHate_node["sign"] = np.arange(1, noHate_node.shape[0] + 1, 1)

n_Hate_node = len(Hate_node)
n_noHate_node = len(noHate_node)

In [ ]:
cols = list(np.arange(0, 1, 0.05))
r = 100

In [ ]:
smax_h_all = pd.DataFrame()
smean_h_all = pd.DataFrame()

for i in cols:
    smax = []
    smean = []

    for j in range(r):
        c_hate = round(n_Hate_node * (i))

        e_h = random.sample(hate_list, c_hate)
        edgelist = weight_edge_all[
            (~weight_edge_all["source"].isin(e_h)) & (~weight_edge_all["target"].isin(e_h))
        ]
        G = nx.from_pandas_edgelist(edgelist, "source", "target")
        s = [len(c) for c in sorted(nx.connected_components(G), key=len, reverse=True)]
        smax.append(max(s) / G.number_of_nodes())
        s.remove(max(s))
        if len(s) != 0:
            smean.append(sum(s) / len(s))
        else:
            smean.append(1)

    smax_h_all[i] = copy.deepcopy(smax)
    smean_h_all[i] = copy.deepcopy(smean)

In [ ]:
smax_n_all = pd.DataFrame()
smean_n_all = pd.DataFrame()

for i in cols:
    smax = []
    smean = []

    for j in range(r):
        c_nohate = round(n_noHate_node * i)

        e_n = random.sample(nohate_list, c_nohate)
        edgelist = weight_edge_all[
            (~weight_edge_all["source"].isin(e_n)) & (~weight_edge_all["target"].isin(e_n))
        ]
        G = nx.from_pandas_edgelist(edgelist, "source", "target")
        s = [len(c) for c in sorted(nx.connected_components(G), key=len, reverse=True)]
        smax.append(max(s) / G.number_of_nodes())
        s.remove(max(s))
        if len(s) != 0:
            smean.append(sum(s) / len(s))
        else:
            smean.append(1)
    smax_n_all[i] = copy.deepcopy(smax)
    smean_n_all[i] = copy.deepcopy(smean)

In [ ]:
index = smax_h_all.columns

In [ ]:
m_smax_h_all = smax_h_all.mean()
std_smax_h_all = smax_h_all.std()
m_smean_h_all = smean_h_all.mean()
std_smean_h_all = smean_h_all.std()

m_smax_n_all = smax_n_all.mean()
std_smax_n_all = smax_n_all.std()
m_smean_n_all = smean_n_all.mean()
std_smean_n_all = smean_n_all.std()

In [ ]:
fig = plt.figure(figsize=(6, 5))
plt.plot(index, m_smean_n_all, "b--", label="hate adjacent <s>")
plt.fill_between(
    index, m_smean_n_all - std_smean_n_all, m_smean_n_all + std_smean_n_all, color="b", alpha=0.2
)
plt.plot(index, m_smean_h_all, "r--", label="hate")
plt.fill_between(
    index, m_smean_h_all - std_smean_h_all, m_smean_h_all + std_smean_h_all, color="r", alpha=0.2
)

plt.legend()
plt.xlim([0, 1])

plt.title("<s>")
plt.xlabel("removed propotion of a type of nodes")
plt.ylabel("<s>")

fig.tight_layout()

In [ ]:
fig = plt.figure(figsize=(6, 5))

plt.plot(index, m_smax_n_all, "b-", label="hate adjacent S")
plt.fill_between(
    index, m_smax_n_all - std_smax_n_all, m_smax_n_all + std_smax_n_all, color="b", alpha=0.2
)
plt.plot(index, m_smax_h_all, "r-", label="hate nodes S")
plt.fill_between(
    index, m_smax_h_all - std_smax_h_all, m_smax_h_all + std_smax_h_all, color="r", alpha=0.2
)
plt.legend()
plt.xlim([0, 1])

plt.title("S")
plt.xlabel("removed propotion of a type of nodes")
plt.ylabel("S")

fig.tight_layout()